# 4. Visualizações - Ames Housing Dataset

Este notebook foca em criar visualizações informativas sobre o dataset, incluindo:
1. Relações entre variáveis numéricas
2. Análise de variáveis categóricas
3. Visualizações geográficas
4. Análise temporal

In [ ]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Configurações de visualização
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
# Carregando os dados tratados
df = pd.read_csv('../data/ames_housing_treated.csv')

## 1. Relações entre Variáveis Numéricas

In [ ]:
# Scatter plots das principais variáveis numéricas vs Preço
numeric_cols = ['GrLivArea', 'TotalBsmtSF', 'GarageArea', 'YearBuilt']

fig = plt.figure(figsize=(15, 10))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(2, 2, i)
    plt.scatter(df[col], df['SalePrice'], alpha=0.5)
    plt.xlabel(col)
    plt.ylabel('Preço de Venda')
    plt.title(f'Preço vs {col}')

plt.tight_layout()
plt.show()

## 2. Análise de Variáveis Categóricas

In [ ]:
def plot_categorical_analysis(df, column):
    plt.figure(figsize=(15, 5))
    
    # Contagem de categorias
    plt.subplot(1, 2, 1)
    sns.countplot(data=df, y=column, order=df[column].value_counts().index[:10])
    plt.title(f'Top 10 Categorias - {column}')
    
    # Boxplot do preço por categoria
    plt.subplot(1, 2, 2)
    sns.boxplot(data=df, y=column, x='SalePrice', 
                order=df.groupby(column)['SalePrice'].median().sort_values(ascending=False).index[:10])
    plt.title(f'Distribuição de Preços por {column}')
    
    plt.tight_layout()
    plt.show()

# Analisando principais variáveis categóricas
categorical_cols = ['Neighborhood', 'HouseStyle', 'ExterQual', 'KitchenQual']
for col in categorical_cols:
    plot_categorical_analysis(df, col)

## 3. Análise Temporal

In [ ]:
# Evolução do preço médio por ano de construção
year_price = df.groupby('YearBuilt')['SalePrice'].agg(['mean', 'count']).reset_index()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=year_price['YearBuilt'],
    y=year_price['mean'],
    mode='lines+markers',
    name='Preço Médio',
    marker=dict(size=year_price['count']/5)  # Tamanho do marcador proporcional à quantidade
))

fig.update_layout(
    title='Evolução do Preço Médio por Ano de Construção',
    xaxis_title='Ano de Construção',
    yaxis_title='Preço Médio',
    showlegend=True
)

fig.show()

## 4. Análise de Características da Casa

In [ ]:
# Criando um gráfico de radar para comparar casas de diferentes faixas de preço
def create_price_categories(price):
    if price < df['SalePrice'].quantile(0.33):
        return 'Baixo'
    elif price < df['SalePrice'].quantile(0.67):
        return 'Médio'
    else:
        return 'Alto'

df['PriceCategory'] = df['SalePrice'].apply(create_price_categories)

# Selecionando características para o gráfico de radar
features = ['GrLivArea', 'TotalBsmtSF', 'GarageArea', 'LotArea']

# Calculando médias por categoria de preço
radar_data = df.groupby('PriceCategory')[features].mean()

# Normalizando os dados
radar_data_norm = (radar_data - radar_data.min()) / (radar_data.max() - radar_data.min())

# Criando o gráfico de radar
fig = go.Figure()

for category in radar_data_norm.index:
    fig.add_trace(go.Scatterpolar(
        r=radar_data_norm.loc[category],
        theta=features,
        fill='toself',
        name=category
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )),
    showlegend=True,
    title='Características da Casa por Faixa de Preço'
)

fig.show()